<a href="https://colab.research.google.com/github/AlexKressner/WS25_Supply_Chain_Optimierung/blob/main/Fallstudie_Juicy_AG_Produktionsplanung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fallstudie Juicy AG, Produktionsplanung
(Capacitated Lot Sizing Problem (CLSP))

In [58]:
! pip install -q pyscipopt

In [59]:
import pandas as pd
from pyscipopt import Model, quicksum

## Repo & Daten laden, Bsp. Produktions- und Transportkosten

In [60]:
! git clone https://github.com/AlexKressner/WS25_Supply_Chain_Optimierung

fatal: destination path 'WS25_Supply_Chain_Optimierung' already exists and is not an empty directory.


In [61]:
folder = "WS25_Supply_Chain_Optimierung/Daten/Produktionsplanung"

In [62]:
# Nachfrage
nachfrage = pd.read_csv(f"{folder}/Nachfrage.csv", sep=";")

In [63]:
nachfrage.tail()

,Produkt,Woche,Nachfrage
20,StraB,1,280
21,StraB,2,0
22,StraB,3,70
23,StraB,4,210
24,StraB,5,980


In [64]:
# Kosten & Zeiten
kosten_zeiten = pd.read_csv(f"{folder}/Kosten_Zeiten.csv", sep=";", decimal=",")

In [65]:
kosten_zeiten

,Produkt,Rüstkosten,Lagerkosten,Rüstzeit,Bearbeitungszeit
0,Well,100,3,420,1
1,BraTee,250,5,700,2
2,HafTee,200,6,560,1
3,DirTee,150,4,560,3
4,StraB,250,3,1400,2


## Indexmengen

In [66]:
P = nachfrage["Produkt"].unique().tolist() # Menge der Produkte

In [67]:
T = nachfrage["Woche"].unique().tolist() # Menge der Planungsperioden (Wochen)

## Modellinstanz ziehen

In [68]:
model = Model()

## Entscheidungsvariablen

In [69]:
infinity = model.infinity()

In [70]:
# Produktionsmengen (Q_pt)
Q = {}
for p in P:
  for t in T:
    Q[p,t] = model.addVar(vtype="I", name=f"Produktionsmenge_Produkt_{p}_in_Woche_{t}", lb=0)


In [71]:
# Lagerbestand
B = {}
for p in P:
  for t in T:
    B[p,t] = model.addVar(vtype="I", name=f"Lagerendbestand_Produkt_{p}_in_Woche_{t}", lb=0)

In [72]:
# Rüstvariablen
R = {}
for p in P:
  for t in T:
    R[p,t] = model.addVar(vtype="B", name=f"Rüstvariable_Produkt_{p}_in_Woche_{t}")

In [73]:
print('Anzahl Entscheidungsvariablen =', len(model.getVars()))

Anzahl Entscheidungsvariablen = 75


In [74]:
len(T)*(len(P)+len(P)+len(P))

75

## Parameter

In [75]:
# Kosten
lc = kosten_zeiten.set_index(["Produkt"]).to_dict("dict")["Lagerkosten"]
rc = kosten_zeiten.set_index(["Produkt"]).to_dict("dict")["Rüstkosten"]

In [76]:
# Zeiten
bz = kosten_zeiten.set_index(["Produkt"]).to_dict("dict")["Bearbeitungszeit"]
rz = kosten_zeiten.set_index(["Produkt"]).to_dict("dict")["Rüstzeit"]

In [77]:
# Nachfrage
d = nachfrage.set_index(["Produkt","Woche"]).to_dict("dict")["Nachfrage"]

In [78]:
# Kapazitäten
cap = {}
for t in T:
  cap[t] = 5760

## Zielfunktion

### Zielfunktion
Min $K = \sum_{p,t} (rc_p * R_{p,t} + lc_p * B_{pt})$

In [79]:
# Minimierung der gesamten Kosten
model.setObjective(
  quicksum(
      rc[p]*R[p,t]
      + lc[p]*B[p,t] for p in P for t in T
  )
    )

## Nebenbedingungen

**(1) Lagerbilanzgleichung**

$B_{pt} = B_{p,t-1} + Q_{pt} - d_{pt}$

$∀ p, t$

In [80]:
for p in P:
  for t in T:
    model.addCons(B[p,t] == B.get((p,t-1),0) + Q[p,t] - d[p,t], name="Lagerbilanzgleichung")

**(2) Produktionskapazitäten**

$\sum_{p} (bz_p * Q_{pt} + rz_p * R_{pt}) \le cap_{t}$

$∀ t$

In [81]:
for t in T:
  model.addCons(quicksum(bz[p]*Q[p,t] + rz[p]*R[p,t]for p in P) <= cap[t])

**(3) Koppelung Produktions- mit Rüstentscheidung**

$Q_{pt} \le \frac{cap_t}{bz_p} * R_{pt} $

$∀ p, t$

In [82]:
for p in P:
  for t in T:
    model.addCons(Q[p,t] <= (cap[t]/bz[p])*R[p,t])

## Berechnung Lösung

In [83]:
model.optimize()

print('LÖSUNG:')
print('Zielfunktionswert (Kosten) =', model.getObjVal())

LÖSUNG:
Zielfunktionswert (Kosten) = 8024.0


In [84]:
for t in T:
  print(f"Woche: {t}")
  produzieren = sum(bz[p] * model.getVal(Q[p,t]) for p in P)
  rüsten = sum(rz[p] * model.getVal(R[p,t]) for p in P)
  print(f"Produktionszeit: {round(produzieren)}")
  print(f"Rüstzeit: {round(rüsten)}")
  print(f"Produzierte Produkte: {[p for p in P if model.getVal(R[p,t])>0.0]}")
  print(f"Summe: {round(rüsten+produzieren)}")

  print("\n")


Woche: 1
Produktionszeit: 910
Rüstzeit: 2380
Produzierte Produkte: ['Well', 'HafTee', 'StraB']
Summe: 3290


Woche: 2
Produktionszeit: 2971
Rüstzeit: 2240
Produzierte Produkte: ['Well', 'BraTee', 'HafTee', 'DirTee']
Summe: 5211


Woche: 3
Produktionszeit: 3520
Rüstzeit: 2240
Produzierte Produkte: ['Well', 'BraTee', 'HafTee', 'DirTee']
Summe: 5760


Woche: 4
Produktionszeit: 3780
Rüstzeit: 1820
Produzierte Produkte: ['Well', 'StraB']
Summe: 5600


Woche: 5
Produktionszeit: 3939
Rüstzeit: 1820
Produzierte Produkte: ['BraTee', 'HafTee', 'DirTee']
Summe: 5759


